In [6]:
import numpy as np
import tensorflow as tf
import time
sess = tf.Session()

In [9]:
a = tf.constant(np.random.rand(30,40,50))
b = tf.constant(np.random.rand(50,40))

In [10]:
start_time = time.clock()
a_len = a.get_shape().as_list()[0]
outputs = []
for i in range(a_len):
    outputs.append(tf.matmul(a[i], b))
output_from_looping = sess.run(outputs)
print(time.clock() - start_time, "seconds")
output_from_looping = np.array(output_from_looping)
print(output_from_looping.shape)

0.2387843479226106 seconds
(30, 40, 40)


In [11]:
start_time = time.clock()
output_from_enisum_tensor = tf.einsum('ijk,kl->ijl',a,b)
output_from_enisum = sess.run(output_from_enisum_tensor)
print(time.clock() - start_time, "seconds")
print(output_from_enisum.shape)

0.0351566568617514 seconds
(30, 40, 40)


In [12]:
np.array_equal(output_from_looping, output_from_enisum)

True

In [ ]:
hidden_size = 80
batch_size = 128
num_attention_features = 4
max_sentences_per_input = 6

b1_val = np.random.rand(hidden_size,)
b2_val = np.random.rand(1)
w1_val = np.random.rand(num_attention_features*hidden_size, hidden_size)
w2_val = np.random.rand(hidden_size, 1)

#dim: [max_sentences_per_input, batch_size, 4*hidden_size]
feature_vectors_val = np.random.rand(max_sentences_per_input,batch_size,num_attention_features*hidden_size)

feature_vectors = tf.constant(feature_vectors_val)
b_1 = tf.constant(b1_val)
b_2 = tf.constant(b2_val)
W_1 = tf.constant(w1_val)
W_2 = tf.constant(w2_val)

In [20]:
output_from_looping = []
start_time = time.clock()
for i in range(max_sentences_per_input):
    output_from_looping.append(tf.matmul(tf.tanh(tf.matmul(feature_vectors[i], W_1) + b_1), W_2) + b_2)
output_from_looping_val = sess.run(output_from_looping)
print(time.clock() - start_time, "seconds")
output_from_looping_val = np.array(output_from_looping_val)
print(output_from_looping_val.shape)

0.11036152987048808 seconds
(6, 128, 1)


In [23]:
#tf.matmul(tf.tanh(tf.matmul(feature_vectors[i], W_1) + b_1), W_2) + b_2
start_time = time.clock()
output_from_matmul_feature_W1 = tf.einsum('ijk,kl->ijl', feature_vectors, W_1)
inner_term = tf.tanh(output_from_matmul_feature_W1 + b_1)
full_term = tf.einsum('ijk,kl->ijl', inner_term, W_2) + b_2
output_from_enisum_val = sess.run(full_term)
print(time.clock() - start_time, "seconds")
print(output_from_enisum_val.shape)

0.05832846128737401 seconds
(6, 128, 1)


In [24]:
np.array_equal(output_from_looping_val, output_from_enisum_val)

True

In [64]:
def matmul_3d_2d(X_3d, W_2d, bias_to_add=None):
    #bias can be 1d or 0d
    out = tf.einsum('ijk,kl->ijl', X_3d, W_2d)
    if bias_to_add is not None:
        out += bias_to_add
    return out

In [67]:
start_time = time.clock()

inner_term = tf.tanh(matmul_3d_2d(feature_vectors, W_1, b_1))
attentions = matmul_3d_2d(inner_term, W_2, b_2)
output_from_enisum_fn_val = sess.run(attentions)

print(time.clock() - start_time, "seconds")
print(np.array_equal(output_from_enisum_val, output_from_enisum_fn_val))
print(output_from_enisum_val.shape)

0.06390198026838334 seconds
True
(6, 128, 1)


In [66]:
np.array_equal(output_from_enisum_val, output_from_enisum_fn_val)

True

In [51]:
a = np.random.rand(20,30,40)
b = np.random.rand(40,1)
b_squeeze = np.squeeze(b)
c = np.matmul(a,b)
d = np.matmul(a, b_squeeze)
np.array_equal(np.squeeze(c), d)

True

In [57]:
#will not work: c_tensor_val = sess.run(tf.matmul(a, b))
c_tensor_einsum_val = sess.run(tf.einsum('ijk,kl->ijl', tf.constant(a), tf.constant(b))+1)
#d_tensor_val = sess.run(tf.matmul(a, b_squeeze))
#np.array_equal(c_tensor_einsum_val, d_tensor_val)